using the bert for embedding and FAISS for the nearest neighbour search

In [2]:
!pip install -q sentence-transformers faiss-cpu pandas

In [3]:
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

c:\Users\Shahana\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# all-MIniLM-L6-v2 

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')  

Combine the text data that has to be embedded into a single column in data frame and then to list .

In [6]:
df = pd.read_csv("PreProcessedData.csv") 
df['text'] = df['name'] + ". " + df['ingredients'] + ". " + df['instructions']+"."+df['tags']

In [7]:
texts=df['text'].tolist()

In [8]:
embeddings = model.encode(texts, show_progress_bar=True, convert_to_numpy=True)

Batches: 100%|██████████| 110/110 [00:09<00:00, 11.52it/s]


The below cell is to scale the nutritions 

In [11]:
from sklearn.preprocessing import MinMaxScaler

nutrients = df[['calories', 'protein', 'carbohydrates','fiber','fat', 'sodium']].fillna(0)
scaler = MinMaxScaler()
nutrient_scaled = scaler.fit_transform(nutrients)


In [12]:
# combining the text embeddings and the scaled nutritions
combined_embeddings = np.hstack((embeddings, nutrient_scaled))

In [13]:
import faiss

dim = combined_embeddings.shape[1] 
index = faiss.IndexFlatL2(dim)
index.add(combined_embeddings)


In [14]:
import pickle 
# Save combined embeddings
np.save("combined_embeddings.npy", combined_embeddings)

# Save scaler
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Save FAISS index
faiss.write_index(index,"faiss_index.bin")

In [15]:
query_text = "oats , banana milk"
query_embed = model.encode([query_text])

# example nutrient goals
query_nutrient = scaler.transform([[550, 30, 35, 15, 10, 300]])  # scale same as train

# combine
query_vector = np.hstack((query_embed, query_nutrient))
D, I = index.search(query_vector.reshape(1, -1), k=3)


c:\Users\Shahana\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [16]:
df1=pd.read_csv("Dataset_combined.csv")

In [17]:
for i in I:
    print(df1.iloc[i])

      Unnamed: 0                            name  \
1476        1476     banana oats smoothie recipe   
2670         714  Sweet and spicy barbecue sauce   
1412        1412        Blueberry Banana Oatmeal   

                                            ingredients  \
1476  ['1 cup chilled and roughly chopped bananas', ...   
2670  ['1 cup no-added-salt tomato purée', '2 tables...   
1412  ['1/2 cup Rolled Oats', '', '1/2 mashed medium...   

                                           instructions  \
1476  For banana oats smoothieTo make banana oats sm...   
2670  ['1 Combine all ingredients in a small saucepa...   
1412  For blueberry banana oatmeal Place oats, mashe...   

                                              nutrition     time  \
1476  {'Energy': '201 cal', 'Protein': '6 g', 'Carbo...   2 Mins   
2670  {'Calories ': '14 cal', 'Kilojoules ': '60 kJ'...  20 mins   
1412  {'Energy': '402 cal', 'Protein': '15.1 g', 'Ca...  15 Mins   

           serving_size                     